1. Sử dụng ngôn ngữ lập trình Python và thư viện sklearn, viết các hàm để thực hiện các mô hình hồi quy tuyến tính sau:
$$
\begin{aligned}
\underset{\beta}{\operatorname{argmin}}\|\mathbf{y} - \mathbf{X}\beta\|_2^2 + \rho\|\beta\|_1 \\
\underset{\beta}{\operatorname{argmin}}\|\mathbf{y} - \mathbf{X}\beta\|_2^2 + \rho\|\beta\|_2^2
\end{aligned}
$$
Khi nhận đầu vào bao gồm $\mathbf{X} \in \mathbb{R}^{n \times p}$, $\mathbf{y} \in \mathbb{R}^n$, và hằng số $\rho \in \mathbb{R}$, các hàm này trả về lời giải tối ưu $\beta$ khi giải quyết loại bài toán hồi quy tuyến tính cụ thể (Lasso hoặc Ridge).\
2. Sử dụng các hàm ở trên, huấn luyện các mô hình hồi quy tuyến tính được chính quy hóa dạng $l_1$ hoặc $l_2$ trên các tập dữ liệu `data_a.csv` và `data_b.csv`. Cột cuối cùng trong mỗi tập dữ liệu là biến mục tiêu $\mathbf{y}$, và các cột còn lại tạo nên ma trận các biến độc lập $\mathbf{X}$. Với mỗi tập dữ liệu, chia thành $50\%$ cho huấn luyện, $25\%$ kiểm thử, và $25\%$ cho kiểm tra. Sử dụng tập kiểm thử để chọn giá trị tối ưu cho các tham số $\rho$ trong dãy $[0,001, 0.01, 0.1, 1, 2]$ cho các mô hình. Tính $\|\mathbf{y} - \mathbf{X}\beta\|_2$ trên tập kiểm tra sử dụng giá trị $\beta$ từ các mô hình hồi quy tuyến tính được chính quy hóa dạng $l_1$, $l_2$ và dạng thông thường ($\rho = 0$). Bằng việc so sánh thêm với mô hình đơn giản nhất là dự đoán sử dụng giá trị trung bình $\mathbf{y}$ của tập huấn luyện và kiểm thử, cho biết phương pháp nào cho kết quả tốt nhất?\
3. Dựa vào các kết quả trên hai tập dữ liệu, lợi ích của các mô hình hồi quy tuyến tính được chính quy hóa là gì? Trường hợp nào chúng ta nên sử dụng phương pháp hồi quy tuyến tính được chính quy hóa so với dạng thông thường?

In [136]:
# Các bạn trình bày bài giải tại đây. Các bạn nên cố gắng sử dụng Markdown trong trường hợp cần thiết!

# Import thư viện

In [1]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Đọc bộ dữ liệu

In [2]:
data_a = pd.read_csv('data_a.csv', header = None)
data_b = pd.read_csv('data_b.csv', header = None)


# Xử lý dữ liệu

In [3]:
alpha_p = [0.001,0.01,0.1,1,2]


In [4]:
def split_data(data):
  X= data.iloc[:, :-1] 
  y= data.iloc[:,-1] 
  
  X_train, X_temp,y_train,y_temp =train_test_split(X,y,test_size=0.5,random_state=42)
  X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

  return X_train, X_val, X_test, y_train,y_val,y_test

In [5]:
x_a_train, x_a_validation, x_a_test, y_a_train, y_a_validation,y_a_test = split_data(data_a)
x_b_train, x_b_validation, x_b_test, y_b_train, y_b_validation,y_b_test = split_data(data_b)

# Tạo các hàm linear regression

#  + linear regression with regularization

In [6]:
def linear_regression_with_regularization(X, Y, alpha, regularization):
    if regularization == 'Lasso':
        model = Lasso(alpha = alpha)
    elif regularization == 'Ridge':
        model = Ridge(alpha = alpha)
    model.fit(X,Y)
    return model

#  + linear regression with regularization

In [7]:
def linear_regression(X,Y):
    model = LinearRegression()
    model.fit(X,Y)
    return model

# Hàm huấn luyện

In [8]:
def train(x_train, y_train, x_validation, y_validation, x_test, y_test, alpha_p):

    y_pred_train_lasso_s = []
    y_pred_validation_lasso_s = [] 
    y_pred_train_ridge_s= []
    y_pred_validation_ridge_s = []
    mse_validation_s = []
    mse_test_s = []
    y_train_pred_s = []
    y_validation_pred_s = []
    mse_lasso_vals = []
    mse_ridge_vals = []
    mse_lasso_tests = []
    mse_ridge_tests = []

    for alpha in alpha_p:
        # Lasso
        lasso_model = linear_regression_with_regularization(x_train, y_train, alpha, 'Lasso') 
        #lasso_models.append(lasso_model)   #  

        y_pred_validation_lasso = lasso_model.predict(x_validation)
        mse_lasso_val = mean_squared_error(y_validation, y_pred_validation_lasso, squared=False)
        mse_lasso_vals.append(mse_lasso_val)           ##### Lasso_val

        y_pred_test_lasso = lasso_model.predict(x_test)
        mse_lasso_test = mean_squared_error(y_test, y_pred_test_lasso, squared=False)
        mse_lasso_tests.append(mse_lasso_test)         ##### Lasso_test
        
        y_pred_train_lasso = lasso_model.predict(x_train)
        
        y_pred_train_lasso_s.append(y_pred_train_lasso.mean())           ##### Lasso_mean_y_train_pred
        y_pred_validation_lasso_s.append(y_pred_validation_lasso.mean()) ##### Lasso_mean_y_val_pred
        
        
        # Ridge
        ridge_model = linear_regression_with_regularization(x_train, y_train, alpha, 'Ridge')
        #ridge_models.append(ridge_model)

        y_pred_validation_ridge = ridge_model.predict(x_validation)
        mse_ridge_val = mean_squared_error(y_validation, y_pred_validation_ridge, squared=False)
        mse_ridge_vals.append(mse_ridge_val)           ##### Ridge_val

        y_pred_test_ridge = ridge_model.predict(x_test)
        mse_ridge_test = mean_squared_error(y_test, y_pred_test_ridge, squared=False)
        mse_ridge_tests.append(mse_ridge_test)         ##### Ridge_test
        
        y_pred_train_ridge = ridge_model.predict(x_train)
        
        y_pred_train_ridge_s.append(y_pred_train_ridge.mean())           ##### Ridge_mean_y_train_pred
        y_pred_validation_ridge_s.append(y_pred_validation_ridge.mean()) ##### Ridge_mean_y_val_pred
        
    linear_model = linear_regression(x_train, y_train)
    
    y_validation_pred = linear_model.predict(x_validation)
    y_test_pred = linear_model.predict(x_test)
    y_train_pred = linear_model.predict(x_train)
    
    mse_validation = mean_squared_error(y_validation, y_validation_pred, squared=False)  
    mse_validation_s.append(mse_validation)     #### LinearRegression_val
    
    mse_test = mean_squared_error(y_test, y_test_pred, squared=False)
    mse_test_s.append(mse_test)    #### LinearRegression_test   
    
    y_train_pred_s.append(y_train_pred.mean())  #### LR_mean_y_train_pred
    y_validation_pred_s.append(y_validation_pred.mean())  #### LR_mean_y_val_pred
    return mse_lasso_vals, mse_lasso_tests, y_pred_train_lasso_s, y_pred_validation_lasso_s, mse_ridge_vals, mse_ridge_tests, y_pred_train_ridge_s, y_pred_validation_ridge_s, mse_validation_s, mse_test_s, y_train_pred_s, y_validation_pred_s

# Thực hiện huấn luyện

# A. Thực nghiệm trên bộ dữ liệu data_a theo yêu cầu 2

In [9]:
result = train(x_a_train, y_a_train, x_a_validation, y_a_validation, x_a_test, y_a_test, alpha_p)

mse_lasso_vals = result[0]
mse_lasso_tests = result[1]
y_pred_train_lasso_s = result[2]
y_pred_validation_lasso_s = result[3]
mse_ridge_vals = result[4]
mse_ridge_tests = result[5]
y_pred_train_ridge_s = result[6]
y_pred_validation_ridge_s = result[7]
mse_validation_s = result[8]
mse_test_s = result[9]
y_train_pred_s = result[10]
y_validation_pred_s = result[11]


In [10]:
print("### LASSO MODEL")
print("Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_a tương ứng với các giá trị alpha là:")
print(alpha_p)
print(mse_lasso_vals)

### LASSO MODEL
Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_a tương ứng với các giá trị alpha là:
[0.001, 0.01, 0.1, 1, 2]
[0.1251241907709637, 0.12389272509182203, 0.2596937777310851, 0.2596937777310851, 0.2596937777310851]


**Từ kết quả trên của model LASSO, nhận thấy giá trị Root Mean Squared Error (RMSE) nhỏ nhất thuộc về alpha = 0.01 => 0.01 chính là giá trị tối ưu của tham số p cho mô hình Lasso**

In [11]:
print("### RIDGE MODEL")
print("Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_a tương ứng với các giá trị alpha là:")
print(alpha_p)
print(mse_ridge_vals)

### RIDGE MODEL
Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_a tương ứng với các giá trị alpha là:
[0.001, 0.01, 0.1, 1, 2]
[0.16390485423807172, 0.1564811229112059, 0.14220413598859552, 0.11609416628726124, 0.11074222501008223]


**Từ kết quả trên của model RIDGE, nhận thấy giá trị Root Mean Squared Error (RMSE) nhỏ nhất thuộc về alpha = 2 => 2 chính là giá trị tối ưu của tham số p cho mô hình Ridge**

**Giá trị $\|\mathbf{y} - \mathbf{X}\beta\|_2$ tức là dạng thông thường Linear Regression ($\rho = 0$) và RMSE trên test của mô hình $l_1$, $l_2$.**

In [12]:

print("LINEAR REGRESSION: (p = 0)", mse_test_s)
print("LINEAR REGRESSION WITH REGULARIZAION: ")
print("     WITH ALPHA: ")
print("       ", alpha_p)
print("         + LASSO: ", mse_lasso_tests)
print("         + RIDGE: ", mse_ridge_tests)

LINEAR REGRESSION: (p = 0) [0.24058201351648714]
LINEAR REGRESSION WITH REGULARIZAION: 
     WITH ALPHA: 
        [0.001, 0.01, 0.1, 1, 2]
         + LASSO:  [0.15864138364420327, 0.17549431894687667, 0.333551018781526, 0.333551018781526, 0.333551018781526]
         + RIDGE:  [0.23936224145462662, 0.22796262125773775, 0.19340471203249698, 0.15655496838257693, 0.1535112104801779]


**Giá trị trung bình của giá trị y dự đoán trên tập huấn luyện (train) và kiểm thử (validation)**

In [13]:
print("LINEAR REGRESSION: ")
print("   y_pred_train mean = ", y_train_pred_s)
print("   y_pred_train mean = ", y_validation_pred_s)
print("LINEAR REGRESSION WITH REGULARIZATION: ")
print("   WITH ALPHA: ")
print("   ", alpha_p)
print("     LASSO MODEL: ")
print("       y_a_train_pred mean = : ", y_pred_train_lasso_s)
print("       y_a_validation_pred mean = : ", y_pred_validation_lasso_s)
print("     RIDGE MODEL: ")
print("       y_a_train_pred mean = : ", y_pred_train_ridge_s)
print("       y_a_validation_pred mean = : ", y_pred_validation_ridge_s)



LINEAR REGRESSION: 
   y_pred_train mean =  [0.33901639344262335]
   y_pred_train mean =  [0.39171716194340317]
LINEAR REGRESSION WITH REGULARIZATION: 
   WITH ALPHA: 
    [0.001, 0.01, 0.1, 1, 2]
     LASSO MODEL: 
       y_a_train_pred mean = :  [0.3390163934426231, 0.33901639344262313, 0.339016393442623, 0.339016393442623, 0.339016393442623]
       y_a_validation_pred mean = :  [0.3509812579667193, 0.3656011994379325, 0.33901639344262313, 0.33901639344262313, 0.33901639344262313]
     RIDGE MODEL: 
       y_a_train_pred mean = :  [0.33901639344262285, 0.3390163934426233, 0.33901639344262313, 0.339016393442623, 0.33901639344262297]
       y_a_validation_pred mean = :  [0.3926081022947556, 0.38684037020382184, 0.3715327424476283, 0.3585146277226181, 0.3565293403494334]


** Nhận xét cho tập data_a.csv: Với kết quả thu được ở trên, đối tới tập dữ liệu data_a thì mô hình Ridge với alpha =2 đều sẽ cho kết quả tốt hơn Lasso và Linear Regression trên cả hai tập kiểm thử và kiểm tra. Nó cho kết quả tốt hơn LASSO model và LINEAR REGRESSION khi dự đoán sử dụng giá trị trung bình 𝐲 của tập huấn luyện và kiểm thử  **

# B. Thực nghiệm trên bộ dữ liệu data_b theo yêu cầu 2

In [14]:
result = train(x_b_train, y_b_train, x_b_validation, y_b_validation, x_b_test, y_b_test, alpha_p)

mse_lasso_vals = result[0]
mse_lasso_tests = result[1]
y_pred_train_lasso_s = result[2]
y_pred_validation_lasso_s = result[3]
mse_ridge_vals = result[4]
mse_ridge_tests = result[5]
y_pred_train_ridge_s = result[6]
y_pred_validation_ridge_s = result[7]
mse_validation_s = result[8]
mse_test_s = result[9]
y_train_pred_s = result[10]
y_validation_pred_s = result[11]


In [15]:
print("### LASSO MODEL")
print("Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_b tương ứng với các giá trị alpha là:")
print(alpha_p)
print(mse_lasso_vals)

### LASSO MODEL
Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_b tương ứng với các giá trị alpha là:
[0.001, 0.01, 0.1, 1, 2]
[4.418185262420349, 4.426982557644045, 4.482922757325381, 4.803861299796965, 5.301736390477789]


**Từ kết quả trên của model LASSO, nhận thấy giá trị Root Mean Squared Error (RMSE) nhỏ nhất thuộc về alpha = 0.001 => 0.001 chính là giá trị tối ưu của tham số p cho mô hình Lasso**

In [16]:
print("### RIDGE MODEL")
print("Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_b tương ứng với các giá trị alpha là:")
print(alpha_p)
print(mse_ridge_vals)

### RIDGE MODEL
Giá trị Root Mean Squared Error (RMSE) thu được trên tập kiểm thử của data_b tương ứng với các giá trị alpha là:
[0.001, 0.01, 0.1, 1, 2]
[4.417663283793894, 4.418050580815581, 4.422199265415187, 4.4533738506539216, 4.4668862005794505]


**Từ kết quả trên của model RIDGE, nhận thấy giá trị Root Mean Squared Error (RMSE) nhỏ nhất thuộc về alpha = 0.001 => 0.001 chính là giá trị tối ưu của tham số p cho mô hình Ridge**

**Giá trị $\|\mathbf{y} - \mathbf{X}\beta\|_2$ tức là dạng thông thường Linear Regression ($\rho = 0$) và RMSE trên test của mô hình $l_1$, $l_2$.**

In [17]:

print("LINEAR REGRESSION: (p = 0)", mse_test_s)
print("LINEAR REGRESSION WITH REGULARIZAION: ")
print("     WITH ALPHA: ")
print("       ", alpha_p)
print("         + LASSO: ", mse_lasso_tests)
print("         + RIDGE: ", mse_ridge_tests)

LINEAR REGRESSION: (p = 0) [5.548949539357637]
LINEAR REGRESSION WITH REGULARIZAION: 
     WITH ALPHA: 
        [0.001, 0.01, 0.1, 1, 2]
         + LASSO:  [5.550356979107798, 5.566430257142291, 5.592786170833481, 5.4958555408877485, 5.878490591859225]
         + RIDGE:  [5.549067958162439, 5.55012468283846, 5.559816111726943, 5.602265234988519, 5.606757656959702]


**Giá trị trung bình của giá trị y dự đoán trên tập huấn luyện (train) và kiểm thử (validation)**

In [18]:
print("LINEAR REGRESSION: ")
print("   y_pred_train mean = ", y_train_pred_s)
print("   y_pred_train mean = ", y_validation_pred_s)
print("LINEAR REGRESSION WITH REGULARIZATION: ")
print("   WITH ALPHA: ")
print("   ", alpha_p)
print("     LASSO MODEL: ")
print("       y_b_train_pred mean = : ", y_pred_train_lasso_s)
print("       y_b_validation_pred mean = : ", y_pred_validation_lasso_s)
print("     RIDGE MODEL: ")
print("       y_b_train_pred mean = : ", y_pred_train_ridge_s)
print("       y_b_validation_pred mean = : ", y_pred_validation_ridge_s)



LINEAR REGRESSION: 
   y_pred_train mean =  [22.748616600790513]
   y_pred_train mean =  [22.289624566971526]
LINEAR REGRESSION WITH REGULARIZATION: 
   WITH ALPHA: 
    [0.001, 0.01, 0.1, 1, 2]
     LASSO MODEL: 
       y_b_train_pred mean = :  [22.74861660079052, 22.748616600790513, 22.748616600790513, 22.748616600790513, 22.748616600790513]
       y_b_validation_pred mean = :  [22.28964064682016, 22.289787457949437, 22.289012256692853, 22.300512297924747, 22.301899931367192]
     RIDGE MODEL: 
       y_b_train_pred mean = :  [22.74861660079052, 22.74861660079052, 22.748616600790513, 22.748616600790516, 22.74861660079052]
       y_b_validation_pred mean = :  [22.289623879942436, 22.28961788852559, 22.289574675995418, 22.289840044832534, 22.290507784438173]


**#Nhận xét cho tập data_b.csv: Với kết quả thu được ở trên, đối tới tập dữ liệu data_a thì mô hình Ridge với alpha =0.001 cho kết quả tốt hơn trên tập kiểm thử và mô hình Lasso với alpha =1 cho kết quả tốt hơn trên tập kiểm tra. Chúng tốt hơn Linear Regression. **

# Yêu cầu 3:

1. Lợi ích của mô hình hồi quy tuyến tính được chính quy hóa:


*  Giảm overfitting: Chính quy hóa giúp hạn chế việc mô hình quá khớp với dữ liệu huấn luyện, từ đó cải thiện khả năng dự đoán trên dữ liệu mới.
*   Giảm độ phức tạp của mô hình: Chính quy hóa giúp làm giảm độ phức tạp của mô hình, khiến cho mô hình thu được có thể được diễn giải và giải thích dễ dàng hơn.
*  Tối ưu hóa hệ số: Chính quy hóa giúp tối ưu hóa hệ số của mô hình, giúp thu được giá trị tối ưu cho các hệ số sai số của mô hình.
*  Cải thiện hiệu suất mô hình. Chính quy hóa giúp cải thiện hiệu suất mô hình trên dữ liệu mới bằng cách giúp mô hình học được các mối quan hệ quan trọng trong dữ liệu và tránh học các mối quan hệ ngẫu nhiên.

2. Các trường hợp sử dụng:
- Sử dụng phương pháp hồi quy tuyến tính được chính quy hóa khi: dữ liệu có nhiều đặc trưng và không có đủ dữ liệu huấn luyện(ít sample) để giảm overfitting, tăng độ chính xác của mô hình, cụ thể là dữ liệu trong file data_a.csv

- Sử dụng phương pháp hồi quy thông thường khi: dữ liệu có số lượng đặc trưng ít và đủ dữ liệu huấn luyện, cụ thể là dữ liệu trong file data_b.csv. Tuy nhiê, chúng ta cũm có thể áp dụng các dạng chính quy hóa để đảm bảo mô hình không quá phức tạp.